In [5]:
import asyncio
import concurrent.futures
import requests
import matplotlib.pyplot as plt

RECEIPT_NUMBER_BASE = "EAC209011"
RANGE_START = 3600
RANGE_END = 4800
CASE_TYPE = 'I-765'
CONCURRENCY_LEVEL = 12


def check_status(receipt_num):
    #print(receipt_num)
    pload = {"appReceiptNum": receipt_num, "initCaseSearch":"CHECK STATUS"}
    r = requests.post('https://egov.uscis.gov/casestatus/mycasestatus.do', data=pload)
    response = r.text.split("\n")
    if CASE_TYPE in response[733]:
        return (receipt_num, response[719].strip())
    else:
        return None


def save_plot(status_list):
    x, y = zip(*status_list)
    plt.scatter(x, y)
    plt.show()
    plt.savefig('UscisStats.png', bbox_inches='tight')

status_list = []

async def main():
    num_digits = len(str(RANGE_END))

    loop = asyncio.get_event_loop()
    i = RANGE_START
    while i < RANGE_END:
        print("processing No."+str(i))
        with concurrent.futures.ThreadPoolExecutor(max_workers=CONCURRENCY_LEVEL) as executor:
            loop = asyncio.get_event_loop()
            futures = [
                loop.run_in_executor(
                    executor,
                    check_status,
                    RECEIPT_NUMBER_BASE + str(i+x).zfill(num_digits)
                )
                for x in range(CONCURRENCY_LEVEL)
            ]
            for t in await asyncio.gather(*futures):
                if t is not None:
                    status_list.append(t)

        i = i+CONCURRENCY_LEVEL if i + CONCURRENCY_LEVEL <= RANGE_END else RANGE_END

    save_plot(status_list)


loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending name='Task-4' coro=<main() running at <ipython-input-5-3a78889d3ef0>:32>>

processing No.3600
3756
3756
processing No.3612
3768
3768
processing No.3624
3780
3780
3792
processing No.3636
3792
